In [1]:
# Export LIMS Data for Demo/Summary
%run -i "viral_lims_export.py"
df_lims = export_df_from_LIMS()

# Summary of LIMS Waste-Water Data Transformation and Import into REDCap
### 2021/11/14
### Alexey Gilman

## Raw LIMS Data Columns:

In [2]:
print(f" Number of columns in raw LIMS dataset: {len(df_lims.columns)}")
[i for i in df_lims.columns]

 Number of columns in raw LIMS dataset: 64


['PHLAccessionNumber',
 'EPAID',
 'WWTPName',
 'State',
 'CountiesServiced',
 'Zipcode',
 'CapacityMGD',
 'PopulationServed',
 'IndustrialInput',
 'StormwaterInput',
 'StorageonSite',
 'SewageTravelTime',
 'SampleCollectDate',
 'SampleCollectTime',
 'FlowRate',
 'CollectionWaterTemp',
 'SampleType',
 'CompositeFreq',
 'SampleMatrix',
 'SampleLocation',
 'SampleLocationSpecified',
 'CollectionStorageTime',
 'CollectionStorageTemp',
 'Pretreatment',
 'PretreatmentSpecify',
 'WWTPComments',
 'ConcentrationMethod',
 'ExtractionMethod',
 'PreConcStorageTime',
 'PreConcStorageTemp',
 'PreExtStorageTime',
 'PreExtStorageTemp',
 'TotConcVol',
 'ExtBlank',
 'RecEffTargetName',
 'RecEffSpikeMatrix',
 'RecEffSpikeConc',
 'PCRTarget',
 'PCRTargetRef',
 'PCRType',
 'LODRef',
 'QuantStanType',
 'StanRef',
 'InhibitionMethod',
 'NumNoTargetControl',
 'ReportingState',
 'pH',
 'Conductivity',
 'TSS',
 'EquivSewageAmt',
 'TestResultDate',
 'SARSCoV2Units',
 'SARSCoV2AvgConc',
 'SARSCoV2StdError',
 'SAR

## Preliminary Transformation: preparation for long-to-wide transformation
    1. Rows with missing sample ID's are removed
    2. Rows where PCRTarget column is NOT "N1" or "N2" are removed.
    
##### WARNING: Anything thats not N1 or N2 in PCRTarget field will be removed!

In [3]:
df_lims = df_lims = drop_null_sample_ID(df_lims).pipe(drop_all_but_N1_N2).pipe(below_lod_to_yes_no)
df_lims[['SubmitterSampleNumber', 'PCRTarget', 'SARSCoV2BelowLOD', 'SARSCoV2AvgConc' ]].head(10)

,SubmitterSampleNumber,PCRTarget,SARSCoV2BelowLOD,SARSCoV2AvgConc
82,210004,N1,Yes,9506.87
83,210004,N2,Yes,30835.54
84,210009,N1,Yes,38439.54
85,210009,N2,Yes,71740.81
86,210001,N1,Yes,222585.26
87,210001,N2,Yes,159718.38
88,210007,N1,Yes,71815.51
89,210007,N2,Yes,47314.22
90,210002,N1,Yes,46533.50
91,210002,N2,Yes,40145.98


## Long-to-wide transformation around PCRTarget

In [4]:
df_lims.pivot(index = 'SubmitterSampleNumber', columns = 'PCRTarget', values = ['SARSCoV2AvgConc','SARSCoV2BelowLOD']).head(10)

SARSCoV2AvgConc            SARSCoV2BelowLOD     
PCRTarget                          N1         N2               N1   N2
SubmitterSampleNumber                                                 
210001                      222585.26  159718.38              Yes  Yes
210002                       46533.50   40145.98              Yes  Yes
210003                         386402     393295              Yes  Yes
210004                        9506.87   30835.54              Yes  Yes
210005                       78735.15   87655.11              Yes  Yes
210006                         280038     501467              Yes  Yes
210007                       71815.51   47314.22              Yes  Yes
210008                         128865      89725              Yes  Yes
210009                       38439.54   71740.81              Yes  Yes
210010                          19388      19302              Yes  Yes

## Clean up post long-to-wide transformation
    1. Rename columns and remove multi-index
    2. Remove all duplicates (columns that are not part of long-to-wide transformation)

In [5]:
df_lims = long_to_wide(df_lims)
df_lims[["N1_SARSCoV2AvgConc", "N2_SARSCoV2AvgConc", "N1_SARSCoV2BelowLOD", "N2_SARSCoV2BelowLOD"]].head(10)

,N1_SARSCoV2AvgConc,N2_SARSCoV2AvgConc,N1_SARSCoV2BelowLOD,N2_SARSCoV2BelowLOD
sample_id,,,,
210001,222585.26,159718.38,Yes,Yes
210002,46533.5,40145.98,Yes,Yes
210003,386402.0,393295.0,Yes,Yes
210004,9506.87,30835.54,Yes,Yes
210005,78735.15,87655.11,Yes,Yes
210006,280038.0,501467.0,Yes,Yes
210007,71815.51,47314.22,Yes,Yes
210008,128865.0,89725.0,Yes,Yes
210009,38439.54,71740.81,Yes,Yes


## Isolate only columns to be imported into REDCap and rename columns to match REDCap Fields
#### Columns Names Dictionay: 

In [6]:
df_lims = rename_lims_columns(df_lims)
dict_lims_column_map

{'SampleCollectDate': 'sample_collect_date',
 'SampleCollectTime': 'sample_collect_time',
 'pH': 'ph',
 'Conductivity': 'conductivity',
 'CollectionWaterTemp': 'collection_water_temp',
 'TSS': 'tss',
 'CollectionStorageTime': 'collection_storage_time',
 'CollectionStorageTemp': 'collection_storage_temp',
 'Pretreatment': 'pretreatment',
 'PretreatmentSpecify': 'pretreatment_specify',
 'EquivSewageAmt': 'equiv_sewage_amt',
 'TestResultDate': 'test_result_date',
 'FlowRate': 'flow_rate',
 'SARSCoV2Units': 'sars_cov2_units',
 'SARSCoV2StdError': 'sars_cov2_std_error',
 'SARSCoV2CI95lo': 'sars_cov2_cl_95_lo',
 'SARSCoV2CI95up': 'sars_cov2_cl_95_up',
 'LODSewage': 'lod_sewage',
 'NTCAmplify': 'ntc_amplify',
 'RecEffSpikeConc': 'rec_eff_percent',
 'InhibitionDetect': 'inhibition_detect',
 'InhibitionAdjust': 'inhibition_adjust',
 'ConcentrationMethod': 'concentration_method',
 'ExtractionMethod': 'extraction_method',
 'PreConcStorageTime': 'pre_conc_storage_time',
 'PreConcStorageTemp': 'pre

## Data Validation: "sample_collect_time" must be in format of "HH:MM", if data does not match this pattern it is changed to NaN (missing value)

##### WARNING: "sample_collection_time" not in format "HH:MM" will be changed to missing value

In [7]:
df_lims["sample_collect_time"].unique()

array(['06:30', '08:00', '08:42', '08:19', '08:45', '07:34', '07:35',
       '00:18', '07:00', '06:55', '08:33', '08:43', '  :', '07:22',
       '07:19', '07:13', '06:45', '07:20', '08:32', '08:48', '08:34',
       '07:24', '07:16', '07:18', '07:40', '08:57', '08:38', '07:33',
       '07:32'], dtype=object)

In [8]:
df_lims = verify_time_field(df_lims)

## Data Validation: convert select columns to numeric values or NaN (missing value)

##### WARNING: for all fields below, presence of any non-numeric character will result in missing value

In [9]:
numeric_clms

['ph',
 'conductivity',
 'collection_water_temp',
 'tss',
 'collection_storage_temp',
 'equiv_sewage_amt',
 'flow_rate',
 'sars_cov2_std_error',
 'sars_cov2_cl_95_lo',
 'sars_cov2_cl_95_up',
 'pre_ext_storage_temp']

In [15]:
df_lims[numeric_clms].tail(10)

,ph,conductivity,collection_water_temp,tss,collection_storage_temp,equiv_sewage_amt,flow_rate,sars_cov2_std_error,sars_cov2_cl_95_lo,sars_cov2_cl_95_up,pre_ext_storage_temp
sample_id,,,,,,,,,,,
210027,None,None,7.3,None,1.0,None,6.305,,,,None
210028,None,None,15.0,None,< 6,None,25.3,,,,None
210029,None,None,15.1,None,< 6,None,24.8,,,,None
210030,None,None,7.0,None,<6,None,29.5,,,,None
210031,None,None,3.6,None,3.6,None,7.7,,,,None
210032,None,None,3.9,None,3.9,None,8.0,,,,None
210033,None,None,4.1,None,4.1,None,8.2,,,,None
210034,None,None,6.1,None,1.0,None,6.381,,,,None
210035,None,None,6.2,None,1.0,None,6.465,,,,None


## Data Validation: Transform select free-text fields to numeric

##### WARNING: for all fields below, digits will be preserved but all text will be removed.

In [16]:
text_to_numeric

['pretreatment_specify',
 'pre_conc_storage_time',
 'pre_ext_storage_time',
 'rec_eff_percent',
 'collection_storage_time',
 'tot_conc_vol']

In [18]:
df_lims = freetext_transform(df_lims)

## Data Validation: Transform select fields to either yes/no or NaN

##### Warning: for all fields below, any value other than yes/no will be changed to missing value

In [19]:
yes_no_clms

['quality_flag',
 'inhibition_adjust',
 'ntc_amplify',
 'pretreatment',
 'inhibition_detect',
 'n1_sars_cov2_below_lod',
 'n2_sars_cov2_below_lod']

In [21]:
df_lims[yes_no_clms].tail(10)

,quality_flag,inhibition_adjust,ntc_amplify,pretreatment,inhibition_detect,n1_sars_cov2_below_lod,n2_sars_cov2_below_lod
sample_id,,,,,,,
210027,No,None,No,No,None,Yes,Yes
210028,No,None,No,No,None,No,No
210029,No,None,No,No,None,No,No
210030,No,None,No,No,None,Yes,Yes
210031,No,None,No,No,None,No,No
210032,No,None,No,No,None,No,No
210033,No,None,No,No,None,Yes,Yes
210034,No,None,No,No,None,No,No
210035,No,None,No,No,None,No,No


## Data Validation: "extraction_method" must be one of the following values in REDCap, anything else will be changed to NaN (missing value)

#### WARNING: currently ALL values in "extraction_method" are changed to NaN because values do not match REDCap available choices.

In [27]:
fields_restricted = (
    redcap_metadata_export(redcap_api_url, redcap_tokens_prod["PID171"])
    .pipe(accepted_redcap_fields)
    )
[i for i in fields_restricted["extraction_method"].keys()]

['qiagen-viral',
 'qiagen-fecal',
 'qiagen',
 'qiagen-rneasy-power',
 'qiagen-powerwater',
 'qiagen-rneasy',
 'qiagen-qiaamp-epoch',
 'promega-ht-tna',
 'promega-ht-auto',
 'promega-manual-tna',
 'promega-ww-largevol-tna',
 'nuclisens-auto-magbead',
 'phenol',
 'chemagic300',
 'trizol-zymomagbeads-zymo',
 '4smethod',
 'zymoquick-r2014',
 'none']

## Data Validation: 'concentration_method' must be one of the following values in REDCap, anything else will be changed to NaN (missing value)

#### WARNING: currently ALL values in "extraction_method" are changed to NaN because values do not match REDCap available choices.

In [32]:
[i for i in fields_restricted['concentration_method'].keys()]

['mf-mgcl2',
 'mf-acid',
 'mf-acid-mgcl2',
 'mf',
 'mf-mgcl2-addsolids',
 'mf-acid-addsolids',
 'mf-acid-mgcl2-addsolids',
 'mf-addsolids',
 'peg',
 'ultracentrifugation',
 'skimmilk',
 'beefextract',
 'promega-tna',
 'uf-centricon',
 'uf-amicon',
 'uf-hf-deadend',
 'uf-innovaprep',
 'noconc-addsolids',
 '13']